# Sztuczne sieci neuronowe - laboratorium 3

In [72]:
import torch

### Pytania kontrolne

1. Jakie znasz funkcje straty (w regresji i w klasyfikacji)?
    1. Regresja:
        1. Mean Squared Error (MSE)
        2. Mean Absolute Error (MAE)
        3. Huber Loss
    2. Klasyfikacja:
        1. Cross-Entropy Loss
        2. Binary Cross-Entropy Loss
        3. Hinge Loss
2. Jakie znasz algorytmy (min. jeden, wraz z wariantami) minimalizacji funkcji straty?
    1. Gradient Descent (GD)
        1. Stochastic Gradient Descent (SGD)
        2. Mini-batch Gradient Descent
        3. Batch Gradient Descent
    2. Adam (Adaptive Moment Estimation)
    3. RMSProp
    4. Adagrad
3. Czym jest gradient funkcji?
    1. Gradient funkcji to wektor zawierający pochodne cząstkowe tej funkcji względem wszystkich jej zmiennych. Wskazuje kierunek najszybszego wzrostu funkcji.
4. Wymień znane Ci hiperparametry stosowane w algorytmie najszybszego spadku.
    1. Learning rate (współczynnik uczenia)
    2. Momentum
    3. Batch size
    4. Number of iterations (liczba iteracji)
    5. Weight decay (regularyzacja)
5. Jakie problemy mogą wystąpić przy nieodpowiednim doborze hiperparametrów tego algorytmu?
    1. Zbyt duży learning rate: może powodować "wybuchanie" gradientów i niestabilność treningu.
    2. Zbyt mały learning rate: może prowadzić do bardzo wolnej konwergencji.
    3. Brak odpowiedniego momentum: może powodować utknięcie w lokalnych minimach.
    4. Zbyt duży batch size: może prowadzić do problemów z pamięcią.
    5. Zbyt mały batch size: może prowadzić do niestabilnych aktualizacji gradientów.

### Przypomnienie poprzedniego ćwiczenia

Poprzednie ćwiczenie:
> Załóżmy, że mamy dwa termometry:
> - jeden mierzy temperaturę w stopniach Celsjusza
> - drugi mierzy temperaturę w nieznanej nam skali, ale jest bardzo ładny i chcemy go powiesić na ścianie
>
> Zanim to zrobimy, chcemy się dowiedzieć, jak przeliczać wskazania drugiego termometru na stopnie Celsjusza.
> Spróbujemy znaleźć "wzór" tego przekształcenia na podstawie pomiarów dokonanych obydwoma termometrami.

W poprzednim ćwiczeniu:
- na podstawie wizualizacji danych wybraliśmy do tego problemu model liniowy
- zaimplementowaliśmy go jako funkcję `model` (dwa parametry: `w` i `b`)
- zdefiniowaliśmy funkcję straty dla regresji liniowej - błąd średniokwadratowy (funkcja `loss_fn`)
- na potrzeby obliczenia gradientu funkcji straty (wektora pochodnych względem parametrów modelu, `dL/dw`) zdefiniowailśmy funkcje obliczające poszczególne komponenty reguły łańcuchowej (`dL/dw = (dL/dtp) * (dtp/dw)`)
- podjęliśmy próbę minimalizacji funkcji straty algorytmem najszybszego spadku
- eksperymentowaliśmy z wartością stałej uczącej (`learning_rate`)
- zauważyliśmy konieczność normalizacji danych
- na podstawie wyznaczonych wartości parametrów znaleźliśmy przekształcenie ze skali Fahrenheita do skali Celsjusza

Dzisiaj dowiemy się, jak niektóre z tych kroków zrealizować "automatycznie" z użyciem PyTorch `autograd` i `torch.optim`.

In [73]:
data_unknown = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
data_celsius = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]

t_u = torch.tensor(data_unknown)
t_c = torch.tensor(data_celsius)


In [74]:
def model(t_u, w, b):
    return w * t_u + b

In [75]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c) ** 2
    return squared_diffs.mean()

### Normalizacja danych

W poprzednim ćwiczeniu, aby algorytm najszybszego spadku nie "wybuchł", należało znormalizować dane wejściowe. Zgodnie z oryginalnym przykładem z książki "Deep Learning with PyTorch" normalizacja polegała na dziesięciokrotnym pomniejszeniu wejść (aby sprowadzić dane do "bezpieczniejszego" przedziału). Ta niestandardowa normalizacja wprowadziła jednak nieco niepotrzebnego chaosu do zajęć.

Dlatego dzisiaj zrobimy inaczej - poddamy dane wejściowe standaryzacji.


#### Ćwiczenie
Dokonaj standaryzacji danych wejściowych i przypisz je do tensora `t_un`.

In [76]:
t_un = (t_u - t_u.mean()) / t_u.std()

### PyTorch autograd

W przypadku regresji liniowej dla jednej zmiennej wejściowej i dwóch parametrów, "ręczne" obliczenie pochodnych nie było szczególnie uciążliwe. Dla bardziej skomplikowanych modeli (a takimi będą sieci neuronowe) warto użyć narzędzia, które może wykonać tę pracę za nas.

`autograd` to wbudowany w PyTorch silnik do obliczania pochodnych funkcji - także złożonych.

Poza przechowywaniem wartości liczbowych, tensory w PyTorch mogą zapamiętywać, poprzez jakie operacje i z których (innych) tensorów powstały, tworząc strukturę grafu obliczeń ("computation graph"). Na podstawie tego grafu `autograd` może następnie obliczyć pochodne względem poszczególnych parametrów modelu.

Wartości pochodnych będą zapisane w atrybucie `.grad` tensora. Aby włączyć działanie `autograd` dla konkretnego tensora (oraz wszystkich tensorów, które z niego powstają w wyniku różnych operacji), należy przy tworzeniu go podać argument `requires_grad=True`.

#### Ćwiczenie
Stwórz tensor `params` zawierający początkowe wartości parametrów: w = 1 i b = 0. Zadbaj o włączenie `autograd` dla tego tensora. Sprawdź wartość `params.grad` zaraz po utworzeniu tensora.

In [77]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
print(params.grad)

None


### Aktualizacja gradientów

Zawartość atrybutu `grad` aktualizuje się po wywołaniu funkcji `loss.backward()` (gdzie `loss` to tensor reprezentujący  funkcję straty po przejściu "w przód" przez graf).

Wyjaśnienie:  
Podczas obliczania funkcji straty (gdy dla `params` ustawimy `requires_grad=True`), poza samymi obliczeniami tworzy się graf reprezentujący poszczególne operacje jako wierzchołki (końcowym wierzchołkiem jest `loss`). Po wywołaniu `loss.backward()` graf przetwarzany jest wstecz i obliczane są pochodne względem poszczególnych parametrów.

#### Ćwiczenie
Użyj przygotowanych funkcji `model` i `loss_fn`, aby stworzyć graf obliczeń i uzyskać tensor reprezentujący funkcję straty, a następnie uruchom na nim metodę `backward()`. Sprawdź wartość `params.grad`. Sprawdź też (np. z użyciem `print`), jaka jest zawartość tensorów funkcji straty i predykcji.

In [78]:
print(model(t_u, *params))
print(loss_fn(model(t_u, *params), t_c).backward())
print(params.grad)

tensor([35.7000, 55.9000, 58.2000, 81.9000, 56.3000, 48.9000, 33.9000, 21.8000,
        48.4000, 60.4000, 68.4000], grad_fn=<AddBackward0>)
None
tensor([4517.2969,   82.6000])


### Akumulacja gradientów
Uwaga:  
Ze względów praktycznych (np. ze względu na zastosowanie w przypadku ograniczonej pamięci), każde wywołanie `loss.backward()` powoduje akumulację  gradientów (dodanie nowo obliczonych do już istniejących wartości w `.grad`, a nie nadpisanie). Zwykle pożądanym zachowaniem jest jednak użycie jedynie gradientów z aktualnej iteracji. W tym celu należy "wyzerować" gradienty, np. na początku każdej iteracji algorytmu optymalizacji.

#### Ćwiczenie
Do wyzerowania gradientów na początku każdej iteracji można użyć funkcji `.zero()`.
Wyzeruj gradienty tensora `params`. Pomyśl, jak zabezpieczyć się przed błędem, który może się pojawić w pierwszej iteracji algorytmu optymalizacji.

Uwaga:  
W PyTorchu obowiązuje konwencja, że funkcje z `_` na końcu nazwy służą do wywoływania operacji "w miejscu" (in-place) - nie tworzą kopii tensora / widoku na pamieć, ale zmieniają jej zawartość. W ćwiczeniu użyj wariantu "in-place" funkcji zerującej tensor.


In [79]:
params.grad.zero_()

tensor([0., 0.])

#### Ćwiczenie
Korzystając z kodu funkcji `training_loop` z poprzedniego ćwiczenia (przeklejonej poniżej), napisz funkcję `training_loop_autograd`, w której ręczne obliczanie gradientu zastąpi użycie silnika `autograd`.

Uwaga:  
Krok aktualizacji tensora parametrów należy dodatkowo zamknąć w bloku `with torch.no_grad()`: (aby "na chwilę" wyłączyć autograd na potrzeby zmiany zawartości tensora `params`).

In [80]:
def training_loop(n_iters, learning_rate, params, t_u, t_c):
    for iteration in range(1, n_iters+1):
        w, b = params
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
        grad = grad_fn(t_u, t_c, t_p, w, b)
        
        # tzw. "vanilla" gradient descent        
        params = params - learning_rate * grad
        
        print('After iteration %d, Loss %f' % (iteration, float(loss)))
    
    return params

In [81]:
def training_loop_autograd(n_iters, learning_rate, params, t_u, t_c):
    for iteration in range(1, n_iters+1):
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
        loss.backward()

        with torch.no_grad():
            params -= learning_rate * params.grad
            params.grad.zero_()

        print('After iteration %d, Loss %f' % (iteration, float(loss)))

    return params

#### Ćwiczenie
Uruchom `training_loop_autograd` dla 1000 iteracji, ustaw `learning_rate = 5e-3` i jako dane wejściowe podaj znormalizowany tensor `t_un`. Zainicjalizuj parametry tensorem [1,0, 0.0] (włącz dla niego autograd).

In [82]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
training_loop_autograd(1000, 5e-3, params, t_un, t_c)

After iteration 1, Loss 171.868347
After iteration 2, Loss 168.612122
After iteration 3, Loss 165.418777
After iteration 4, Loss 162.287109
After iteration 5, Loss 159.215927
After iteration 6, Loss 156.204025
After iteration 7, Loss 153.250275
After iteration 8, Loss 150.353577
After iteration 9, Loss 147.512817
After iteration 10, Loss 144.726883
After iteration 11, Loss 141.994751
After iteration 12, Loss 139.315399
After iteration 13, Loss 136.687714
After iteration 14, Loss 134.110779
After iteration 15, Loss 131.583588
After iteration 16, Loss 129.105179
After iteration 17, Loss 126.674629
After iteration 18, Loss 124.290962
After iteration 19, Loss 121.953300
After iteration 20, Loss 119.660767
After iteration 21, Loss 117.412483
After iteration 22, Loss 115.207573
After iteration 23, Loss 113.045235
After iteration 24, Loss 110.924606
After iteration 25, Loss 108.844902
After iteration 26, Loss 106.805344
After iteration 27, Loss 104.805122
After iteration 28, Loss 102.843498
A

tensor([ 9.0340, 10.4995], requires_grad=True)

#### Ćwiczenie
Zapisz równanie transformujące oryginalne dane wejściowe (`t_u`) do stopni Celsjusza (`t_p`).
Znajdź współczynniki prostej opisującej model liniowy. Porównaj z wynikiem z poprzednich zajęć.

Uwaga / hint:
Uwzględnij średnią i odchylenie standardowe policzone dla danych wejściowych na początku dzisiejszych zajęć.

In [83]:
w, b = params.detach().numpy()

mean_t_u = t_u.mean().item()
std_t_u = t_u.std().item()

w_transformed = w / std_t_u
b_transformed = b - (w * mean_t_u / std_t_u)

print(f"Równanie transformujące: t_p = {w_transformed} * t_u + {b_transformed}")

Równanie transformujące: t_p = 0.5367202758789062 * t_u + -17.30256462097168


In [84]:
print("Równanie z poprzednich zajęć: t_p = 0.1 * 5.367 * t_u + 32.0")

Równanie z poprzednich zajęć: t_p = 0.1 * 5.367 * t_u + 32.0


### torch.optim

Algorytm najszybszego spadku jest przykładem jednego z wielu algorytmów optymalizacji dostępnych w PyTorch. Implementacje różnych optymalizatorów znajdują się w module `torch.optim`.

Przykłady optymalizatorów w PyTorch:
- `SGD` - Stochastic Gradient Descent (tak naprawdę tzw. Batch Gradient Descent), z dodatkową opcją tzw. "momentum"
- `Adagrad`, `RMSProp` - "adaptive, per-parameter learning rate"
- `Adam` - Adaptive Moment Estimation - połączenie SGD+momentum z RMSProp - obecnie bardzo popularny
- ...

Bardziej szczegółowo omówimy te algorytmy w późniejszym czasie.

In [103]:
import torch.optim as optim
dir(optim)

['ASGD',
 'Adadelta',
 'Adafactor',
 'Adagrad',
 'Adam',
 'AdamW',
 'Adamax',
 'LBFGS',
 'NAdam',
 'Optimizer',
 'RAdam',
 'RMSprop',
 'Rprop',
 'SGD',
 'SparseAdam',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_adafactor',
 '_functional',
 'lr_scheduler',
 'swa_utils']

### Optymalizacja funkcji straty w torch.optim

Optymalizator inicjalizuje się wybierając konkretną klasę z dostępnych w `torch.optim`. Pierwszym argumentem przy tworzeniu optymalizatora jest lista parametrów modelu (nie zawsze wszystkich - czasem np. "zamraża" się niektóre parametry i nie aktualizuje ich). Tworząc obiekt optymalizatora podaje się też jako argument m.in. wartość learning rate (argument `lr`).

W API optymalizatora są dwie metody: `zero_grad()` oraz `step()`. Pierwsza z nich zeruje gradienty parametrów (patrz wyżej), a druga wykonuje krok aktualizacji parametrów zgodny z wybranym algorytmem.

#### Ćwiczenie
Zainicjalizuj tensor `params` jako [1.0, 0.0] (z włączonym autograd) oraz optymalizator SGD ze stałą uczącą o wartości 0.01.


In [121]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
optim_SGD = optim.SGD([params], lr=0.01)

#### Ćwiczenie
Na podstawie `training_loop_autograd` (napisanej wyżej) napisz funkcję `training_loop_optim`, w której zerowanie gradientów i krok aktualizacji parametrów wykonane będą z użyciem optymalizatora z `torch.optim`.

In [122]:
def training_loop_optim(n_iters, optim, params, t_u, t_c):
    for iteration in range(1, n_iters+1):
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
        loss.backward()

        optim.step()
        optim.zero_grad()

        print('After iteration %d, Loss %f' % (iteration, float(loss)))

    return params

#### Ćwiczenie
Uruchom `training_loop_optim` i porównaj wynik z `training_loop_autograd`. Powinno wyjść to samo.

In [123]:
training_loop_optim(1000, optim_SGD,params, t_un, t_c)

After iteration 1, Loss 171.868347
After iteration 2, Loss 165.387650
After iteration 3, Loss 159.156021
After iteration 4, Loss 153.163925
After iteration 5, Loss 147.402084
After iteration 6, Loss 141.861664
After iteration 7, Loss 136.534119
After iteration 8, Loss 131.411301
After iteration 9, Loss 126.485237
After iteration 10, Loss 121.748444
After iteration 11, Loss 117.193611
After iteration 12, Loss 112.813721
After iteration 13, Loss 108.602028
After iteration 14, Loss 104.552116
After iteration 15, Loss 100.657707
After iteration 16, Loss 96.912842
After iteration 17, Loss 93.311745
After iteration 18, Loss 89.848923
After iteration 19, Loss 86.519012
After iteration 20, Loss 83.316925
After iteration 21, Loss 80.237762
After iteration 22, Loss 77.276772
After iteration 23, Loss 74.429413
After iteration 24, Loss 71.691315
After iteration 25, Loss 69.058289
After iteration 26, Loss 66.526276
After iteration 27, Loss 64.091408
After iteration 28, Loss 61.749950
After iteratio

tensor([ 9.0349, 10.5000], requires_grad=True)

#### Ćwiczenie
Sprawdź, jak w porównaniu do `SGD` poradzi sobie optymalizator `Adam` dla `learning_rate = 0.01` dla nieznormalizowanego (oryginalnego) tensora danych wejściowych `t_u`.

In [124]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
optim_Adam = optim.Adam([params], lr=0.01)
training_loop_optim(1000, optim_Adam, params, t_u, t_c)

After iteration 1, Loss 1763.884766
After iteration 2, Loss 1718.190308
After iteration 3, Loss 1673.121216
After iteration 4, Loss 1628.688721
After iteration 5, Loss 1584.901978
After iteration 6, Loss 1541.771606
After iteration 7, Loss 1499.305176
After iteration 8, Loss 1457.512085
After iteration 9, Loss 1416.400391
After iteration 10, Loss 1375.976685
After iteration 11, Loss 1336.247925
After iteration 12, Loss 1297.220581
After iteration 13, Loss 1258.899414
After iteration 14, Loss 1221.289429
After iteration 15, Loss 1184.394653
After iteration 16, Loss 1148.218384
After iteration 17, Loss 1112.763184
After iteration 18, Loss 1078.031250
After iteration 19, Loss 1044.023438
After iteration 20, Loss 1010.740662
After iteration 21, Loss 978.182800
After iteration 22, Loss 946.348877
After iteration 23, Loss 915.237366
After iteration 24, Loss 884.846436
After iteration 25, Loss 855.173401
After iteration 26, Loss 826.214478
After iteration 27, Loss 797.966370
After iteration 2

tensor([ 0.2700, -2.1840], requires_grad=True)

### Podsumowanie:
- zamiast obliczać gradienty "ręcznie", możemy użyć silnika `autograd`
- tensory korzystające z `autograd` (`requires_grad=True`) posiadają atrybut `grad`
- wykonywanie operacji na tensorach powoduje tworzenie grafu obliczeń (computation graph)
- wywołanie funkcji `loss.backward()` na tensorze reprezentującym funkcję straty powoduje aktualizację gradientów (zmienia się zawartość `.grad` dla tensorów w grafie)
- ze względu na akumulację gradientów typowo należy je wyzerować np. na początku każdej iteracji treningu
- moduł `torch.optim` zawiera implementacje algorytmów optymalizacji funkcji straty, m.in. SGD, RMSProp, Adam
- przy inicjalizacji optymalizatora należy podać jako argument zestaw parametrów modelu do optymalizacji oraz wartość stałej uczącej
- korzystając z `torch.optim` w pętli treningowej należy wywołać `optim.zero_grad()` (zerowanie gradientów) oraz `optim.step()` (krok aktualizacji parametrów)

Podczas poprzednich i dzisiejszych ćwiczeń poznaliśmy tak naprawdę podstawowy wariant mechanizmu uczenia sieci neuronowych. Sieci neuronowe to po prostu bardziej złożone modele niż model liniowy, posiadają one dużo więcej parametrów i są modelami nieliniowymi. W sieciach neuronowych w PyTorch także tworzy się graf obliczeń na tensorach, który służy następnie do obliczania gradientów funkcji straty z użyciem `autograd`. Krok aktualizacji parametrów wykonuje się typowo z użyciem jednego z dostępnych w `torch.optim` algorytmów.

### Wnioski
Zajęcia laboratoryjne przygotowały mnie do korzystania autograd. Poznałem również podstawowe algorytmy optymalizacji funkcji straty, takie jak SGD czy Adam. Dzięki temu będę w stanie wydajniej trenować sieci neuronowe.